In [7]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.test import SMA

# ----------------------
# Download the Data
# ----------------------
df_raw = yf.download(["SPY", "UPRO"], start="2010-01-01", end="2023-01-01", progress=False)

# ------------------------------
# Extract SPY & UPRO Using .xs()
# ------------------------------
df_spy = df_raw.xs('SPY',  level=1, axis=1)
df_upro = df_raw.xs('UPRO', level=1, axis=1)

# -------------------------------
# Rename Columns Correctly
# -------------------------------
df_spy.columns = ['Open_SPY', 'High_SPY', 'Low_SPY', 'Close_SPY', 'Volume_SPY']

# ---------------------------
# Merge SPY and UPRO Data
# ---------------------------
df_merged = pd.concat([df_spy, df_upro], axis=1)
df_merged.dropna(inplace=True)

df_merged.reset_index(inplace=True)  # Ensure the Date index is a normal column if needed


In [8]:
df_merged

,Date,Open_SPY,High_SPY,Low_SPY,Close_SPY,Volume_SPY,Close,High,Low,Open,Volume
0,2010-01-04,86.026451,86.071994,84.644927,85.297736,118944600,2.092603,2.097363,2.043147,2.044205,102592800
1,2010-01-05,86.254189,86.292145,85.662108,85.973333,111579900,2.110586,2.114024,2.070123,2.091544,108986400
2,2010-01-06,86.314903,86.527445,86.102362,86.170676,116074400,2.116669,2.129629,2.099611,2.104239,107920800
3,2010-01-07,86.679268,86.785539,85.912596,86.155501,131091100,2.142454,2.148801,2.086122,2.103974,76939200
4,2010-01-08,86.967705,87.005661,86.276946,86.451531,126402800,2.163347,2.165463,2.112173,2.123941,89388000
...,...,...,...,...,...,...,...,...,...,...,...
3267,2022-12-23,372.400543,372.546420,367.654476,369.230009,59857300,32.361805,32.420682,31.174487,31.576801,12097000
3268,2022-12-27,370.931946,372.633915,369.229977,372.283810,51638200,31.998747,32.440313,31.547370,32.361810,8897400
3269,2022-12-28,366.322052,372.867349,366.088649,370.863861,70911500,30.801611,32.479557,30.742736,31.979118,11445900
3270,2022-12-29,372.915985,373.801013,368.675636,369.210558,66970900,32.371616,32.626741,31.341297,31.478674,14159700


In [11]:
# -------------------------------------
# Define the Strategy
# -------------------------------------
class LeveragedForTheLongRun(Strategy):
    n = 100  # SMA period

    def init(self):
        # Use SPY's Close price for the SMA calculation
        self.sma_underlying = self.I(SMA, self.data.df['Close_SPY'], self.n)

    def next(self):
        spy_close = self.data.df['Close_SPY'].iloc[self.data.index[-1]]
        sma_value = self.sma_underlying[-1]  # Latest SMA value

        # If SPY > SMA, Buy UPRO; Otherwise, close position
        if spy_close > sma_value:
            if not self.position:
                self.buy()
        else:
            if self.position:
                self.position.close()

# ------------------------------------------
# Run the Backtest
# ------------------------------------------
bt = Backtest(
    df_merged,
    LeveragedForTheLongRun,
    cash=100_000,
    commission=.0005,
    trade_on_close=False
)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_6584\649758606.py:26: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(


In [12]:
stats = bt.run()
print(stats)

Start                                     0.0
End                                    3271.0
Duration                               3271.0
Exposure Time [%]                   76.619804
Equity Final [$]                671874.138932
Equity Peak [$]                1470091.737227
Return [%]                         571.874139
Buy & Hold Return [%]             1436.638665
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -54.338179
Avg. Drawdown [%]                   -4.644909
Max. Drawdown Duration                  537.0
Avg. Drawdown Duration              23.896825
# Trades                                 65.0
Win Rate [%]                        38.461538
Best Trade [%]                     113.854957
Worst Trade [%]                    -14.784804
Avg. Trade [%]                    

In [13]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1306', ...)